### Statistiques concernant les cararactéristiques des incidents ayant fait des victimes : comment expliquer le nombre de victimes en fonction de l'incident ?

In [6]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

,Unnamed: 0,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop
232,232,Santa Barbara,California,CA,27286,27563,Santa Barbara,"MULTIPOLYGON (((-119.052774 33.46586, -119.046...",6083.0,6.9,6.0,5.6,5.6,4.4,5.3,60803.0,62116.0,63049.0,66360.0,70651.0,75646.0,434.503,438.921,442.051,444.112,444.998,445.137,32932.0,34201.0,35840.0,36850.0,36437.0,36038.0,31.3,31.4,32.2,32.2,33.3,33.8,5.1,1368295,1474616,1761079,2377167,1570110,1297132,0.609655,0.327236
2946,2946,Richmond,Virginia,VA,27330,30307,Richmond,"POLYGON ((-76.938036 38.085432, -76.93619 38.0...",51159.0,5.9,5.2,4.3,4.3,3.7,4.3,39249.0,42074.0,40161.0,42336.0,46073.0,48747.0,214.020,217.238,221.084,225.669,227.620,229.621,53904.0,48494.0,45300.0,42877.0,38945.0,37323.0,34.8,35.4,36.0,36.7,37.5,38.5,2.0,498426,419764,462997,528369,520827,476760,0.368940,0.606135
1788,1788,Bergen,New Jersey,NJ,27312,29137,Bergen,"POLYGON ((-74.270658 41.021032999999996, -74.2...",34003.0,5.3,4.5,3.6,3.5,3.2,3.8,81615.0,84309.0,88512.0,93205.0,93805.0,100181.0,920.021,923.475,926.481,928.381,932.449,932.897,36890.0,38324.0,39975.0,38806.0,35556.0,31900.0,45.9,46.1,46.9,47.6,47.9,48.3,5.0,120071,92320,90034,121853,103739,93124,0.544974,0.425091
1972,1972,Pasquotank,North Carolina,NC,27315,29324,Pasquotank,"POLYGON ((-76.49133599999999 36.510677, -76.43...",37139.0,7.9,7.3,6.8,5.8,5.2,5.8,40094.0,44596.0,42171.0,43687.0,46709.0,46355.0,39.587,39.479,39.433,39.418,39.420,39.693,7991.0,8313.0,8106.0,7647.0,7467.0,6959.0,20.1,18.9,19.4,18.9,20.4,20.5,3.6,574622,1182349,531876,601445,537813,529916,0.498408,0.476421
1371,1371,Meeker,Minnesota,MN,27305,28714,Meeker,"POLYGON ((-94.763353 45.26455, -94.76334 45.26...",27093.0,5.6,5.0,5.3,5.1,4.3,5.3,55327.0,54354.0,58607.0,61374.0,56068.0,66557.0,23.091,23.129,23.048,23.081,23.037,23.122,1907.0,1695.0,1534.0,1531.0,1424.0,1251.0,17.7,18.0,18.9,18.9,19.1,18.8,3.7,525774,481122,519244,651599,683544,604078,0.261682,0.664289


In [9]:
gun_violence_db = pd.read_csv('df_gun_violence.csv')
gun_violence_db.sample(5)

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
65999,64973,326768,2015-04-17,Iowa,Waterloo,520 Elm Street,0,0,http://www.gunviolencearchive.org/incident/326768,http://www.kcrg.com/subject/news/waterloo-poli...,False,1.0,{0: 'Unknown'},{0: 'Handgun'},"['Non-Shooting Incident', 'Possession (gun(s) ...",42.4960,NaN,-92.3236,1.0,NaN,{0: '21'},{0: 'Adult 18+'},{0: 'Male'},0::Kristopher Spates Jr.,NaN,"{0: 'Unharmed, Arrested'}",{0: 'Subject-Suspect'},NaN,62.0,31.0,0,0,0,0,0,Handgun,NaN,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0
193451,192937,872956,2017-06-17,California,Long Beach,2600 block of Long Beach Blvd,0,0,http://www.gunviolencearchive.org/incident/872956,http://www.longbeach.gov/uploadedImages/Pages/...,False,47.0,{0: 'Unknown'},{0: 'Unknown'},"['Non-Shooting Incident', 'Drug involvement', ...",33.8048,NaN,-118.1890,1.0,fireworks investigation leads to discovery of ...,{0: '35'},{0: 'Adult 18+'},{0: 'Male'},0::Sorvath Ke,NaN,"{0: 'Unharmed, Arrested'}",{0: 'Subject-Suspect'},['http://www.longbeach.gov/police/press-releas...,70.0,33.0,0,0,0,0,0,Unknown,NaN,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
74335,72207,354438,2015-06-09,Texas,El Paso (county),NaN,1,1,http://www.gunviolencearchive.org/incident/354438,http://www.ksla.com/story/29252843/gunman-who-...,False,16.0,NaN,NaN,"['Shot - Dead (murder, accidental, suicide)', ...",31.7776,NaN,-106.4420,NaN,"Martinez shot Hernandez in Anthony, NM and the...","{0: '62', 1: '41'}","{0: 'Adult 18+', 1: 'Adult 18+'}","{0: 'Male', 1: 'Male'}",0::Benjamin Martinez||1::Javier Hernandez,NaN,"{0: 'Killed', 1: 'Injured'}","{0: 'Subject-Suspect', 1: 'Victim'}",NaN,76.0,29.0,1,0,0,0,1,NaN,NaN,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
188313,187468,849652,2017-05-20,Virginia,Petersburg,100 block of Perry St,0,1,http://www.gunviolencearchive.org/incident/849652,http://wric.com/2017/05/20/petersburg-juvenile...,False,3.0,{0: 'Unknown'},{0: 'Unknown'},"['Shot - Wounded/Injured', 'Accidental Shootin...",37.2252,NaN,-77.4089,1.0,one of two juveniles shot while playing with p...,NaN,{0: 'Teen 12-17'},NaN,NaN,NaN,{0: 'Injured'},{0: 'Victim'},NaN,63.0,16.0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
237566,237718,1074266,2018-03-18,California,Watsonville,Sill Rd,0,1,http://www.gunviolencearchive.org/incident/107...,https://www.montereysheriff.org/mcsologs/DPL.PDF,False,NaN,{0: 'Unknown'},{0: 'Unknown'},NaN,NaN,NaN,NaN,1.0,NaN,{0: '23'},{0: 'Adult 18+'},{0: 'Male'},NaN,NaN,{0: 'Injured'},{0: 'Victim'},NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [10]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [11]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']


In [12]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [13]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214

In [14]:
total_pop=total_db_geo.groupby('Etat').agg({'resident_pop_year_2015':'sum'}).reset_index()
total_pop.rename(columns={'resident_pop_year_2015':'pop_state_2015'}, inplace=True)
total_pop

,Etat,pop_state_2015
0,Alabama,4854.803
1,Arizona,6832.810
2,Arkansas,2979.732
3,California,38904.296
4,Colorado,5454.328
5,Connecticut,3588.561
6,Delaware,942.065
7,District of Columbia,677.014
8,Florida,20219.111
9,Georgia,10183.353


In [15]:
total_db_geo=total_db_geo.merge(total_pop, on ='Etat', how='left')

In [16]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

In [17]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')

In [18]:
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')

In [19]:
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

,id_county,geometry
0,27336,"POLYGON ((-86.92119 32.65754, -86.92035 32.658..."
1,27337,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,27338,"POLYGON ((-85.74803 31.61918, -85.74543 31.618..."
3,27339,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,27340,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."


In [20]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,geometry_x,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,pop_state_2015,geometry_y
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,NaN,NaN,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,NaN,NaN,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterf

In [21]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,point_geometry,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,pop_state_2015,polygon_geometry
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,NaN,NaN,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,NaN,NaN,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at 

In [22]:
base_totale=merge_geo.copy()

## Après cet import et traitement rapide des données, on peut passer aux statistiques

In [23]:
print("I. Les dates des incidents peuvent-elles avoir un effet sur les conséquences de celui-ci ?\n \nNous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.")

I. Les dates des incidents peuvent-elles avoir un effet sur les conséquences de celui-ci ?.
 
Nous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.


In [24]:
base_totale['date']=pd.to_datetime(base_totale['date'])

In [25]:
grouped_data = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()

fig1 = px.bar(
    grouped_data,
    x=grouped_data['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig1.update_layout(barmode='group')  # Display bars side by side

fig1.show()


In [26]:
print("Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. \nAinsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.")

Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. 
Ainsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.


In [27]:
grouped_data2 = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()

grouped_data2=grouped_data2.loc[grouped_data2['date'].isin([2014,2015,2016,2017])]

fig2 = px.bar(
    grouped_data2,
    x=grouped_data2['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Moyenne de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig2.update_layout(barmode='group')  # Display bars side by side

fig2.show()

In [28]:
print("Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.")

Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.


In [29]:
grouped_data3 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig3 = px.bar(
    grouped_data3,
    x=grouped_data3['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "Mois", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par mois (cumulé de 2014 à 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig3.update_layout(barmode='group')  # Display bars side by side

fig3.show()



In [30]:
grouped_data4 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig4 = px.bar(
    grouped_data4,
    x=grouped_data4['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "mois", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Moyenne de victimes d'incident par arme à feu par mois (entre 2014 et 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig4.update_layout(barmode='group')  # Display bars side by side

fig4.show()

In [31]:
print("On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. \nDe plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. \nIl semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.")

On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. 
De plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. 
Il semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.


In [32]:
print("\nII. Le type d'armes utilisé influe-t-il le nombre de victimes lors d'un incident ?\n")


II. Le type d'armes utilisé influe-t-il le nombre de victimes lors d'un incident ?



In [44]:
grouped_data5 = base_totale.groupby('gun_type').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig5 = px.bar(
    grouped_data5,
    x=grouped_data5['gun_type'],
    y=['n_injured', 'n_killed'],
    labels={"gun_type": "Arme", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de l'arme à feu",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},log_y=True
)

fig5.update_layout(barmode='group')  # Display bars side by side

fig5.show()

In [75]:
print("Ce graphique, en échelle logarithmique pour la lisibilité, nous montre que, de manière générale, les armes utilisées font un nombre similaire de blessés et de morts. Il n'y a donc pas d'arme qui ne fasse que des blessés mais pas de mort et inversement. \nDe plus, les quatre types d'armes faisant le plus de victimes sont les mêmes que celles présentes dans le plus d'incidents (Handgun, Rifle, Shotgun, 9mm).")

Ce graphique, en échelle logarithmique pour la lisibilité, nous montre que, de manière générale, les armes utilisées font un nombre similaire de blessés et de morts. Il n'y a donc pas d'arme qui ne fasse que des blessés mais pas de mort et inversement. 
De plus, les quatre types d'armes faisant le plus de victimes sont les mêmes que celles présentes dans le plus d'incidents (Handgun, Rifle, Shotgun, 9mm).


In [45]:
grouped_data6 = base_totale.groupby('gun_type').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig6 = px.bar(
    grouped_data6,
    x=grouped_data6['gun_type'],
    y=['n_injured', 'n_killed'],
    labels={"gun_type": "Arme", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident en fonction de l'arme à feu",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig6.update_layout(barmode='group')  # Display bars side by side

fig6.show()

In [42]:
print("On remarque ici que certaines armes, qui ne faisaient pas partie des plus utilisées, font beaucoup plus de blessés et de morts en moyenne que d'autres. \nCes outliers sont : le calibre d'arme .30-06 Springfield qui a fait en moyenne 0.19 blessés par incident (l'échantillon est néanmoins assez peu représentatif puisque cela concerne 3 victimes), \nla munition.30-30 Winchester qui est une cartouche de chasse à percussion et a fait 0.15 mort en moyenne (encore une fois assez peu significatif car concerne 5 victimes), \nla munition.308 Winchester et le 16 gauge qui semble être un type de 'Shotgun'.")
print("Concernant les types d'armes qui étaient significatives en termes de nombre de victimes, on remarque que celle qui fait le plus de blessés et de morts en moyenne est la carabine ('Rifle'), suivie du fusil de chasse ('Shotgun'). \nCes deux armes sont des armes militaires ou de chasse, elles sont donc probablement possédées par des personnes plus à l'aise avec les armes ou plus puissantes, ce qui pourrait expliquer qu'elles fassent plus de victimes. \nL'arme de poing fait mois de blessés et morts en moyenne que ces deux armes précédentes et le '9mm' fait particulièrement peu de victimes en moyenne en comparaison avec les autres. \nIl ne faut pas oublier la catégorie 'arme inconnue' qui fait plus de victimes en moyenne que la carabine et qui représente un échantillon bien plus grand.")

On remarque ici que certaines armes, qui ne faisaient pas partie des plus utilisées, font beaucoup plus de blessés et de morts en moyenne que d'autres. 
Ces outliers sont : le calibre d'arme .30-06 Springfield qui a fait en moyenne 0.19 blessés par incident (l'échantillon est néanmoins assez peu représentatif puisque cela concerne 3 victimes), 
la munition.30-30 Winchester qui est une cartouche de chasse à percussion et a fait 0.15 mort en moyenne (encore une fois assez peu significatif car concerne 5 victimes), 
la munition.308 Winchester et le 16 gauge qui semble être un type de 'Shotgun'.
Concernant les types d'armes qui étaient significatives en termes de nombre de victimes, on remarque que celle qui fait le plus de blessés et de morts en moyenne est la carabine ('Rifle'), suivie du fusil de chasse ('Shotgun'). 
Ces deux armes sont des armes militaires ou de chasse, elles sont donc probablement possédées par des personnes plus à l'aise avec les armes ou plus puissantes, ce qui pour

In [49]:
print("\nIII. Les caractéristiques du suspect peuvent-elles avoir un effet sur les conséquences de l'incident ?\n")


III. Les caractéristiques du suspect peuvent-elles avoir un effet sur les conséquences de l'incident ?



In [46]:
grouped_data7 = base_totale.groupby('young_sus').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig7 = px.bar(
    grouped_data7,
    x=grouped_data7['young_sus'],
    y=['n_injured', 'n_killed'],
    labels={"young_sus": "Suspect entre 18 et 25 ans (1)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de l'âge du suspect",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig7.update_layout(barmode='group')  # Display bars side by side

fig7.show()

In [86]:
print("Lorsque le suspect est jeune, entre 18 et 25 ans, il provoque en moyenne plus de morts lors d'un incident par arme à feu que lorsqu'il appartient à une autre catégorie d'âge (0.28 contre 0.24). \nNéanmoins, un suspect entre 18 et 25 ans fait, en moyenne, moins de blessés qu'un autre type de suspect (0.38 contre 0.53). ")

Lorsque le suspect est jeune, entre 18 et 25 ans, il provoque en moyenne plus de morts lors d'un incident par arme à feu que lorsqu'il appartient à une autre catégorie d'âge (0.28 contre 0.24). 
Néanmoins, un suspect entre 18 et 25 ans fait, en moyenne, moins de blessés qu'un autre type de suspect (0.38 contre 0.53). 


In [50]:
print("\nIV. Les circonstances de l'incident par arme à feu influencent-elles ses conséquences ?\n")


IV. Les circonstances de l'incident par arme à feu influencent-elles ses conséquences ?



In [47]:
grouped_data8 = base_totale.groupby('relation').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig8 = px.bar(
    grouped_data8,
    x=grouped_data8['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig8.update_layout(barmode='group')  # Display bars side by side

fig8.show()

In [90]:
print("Ce graphique mets en évidence de fortes disparités de conséquences selon les différents types d'incidents par arme à feu, ici caractérisés par les différentes relation du suspect aux victimes. \nEn effet, on remarque que le type d'incident provoquant, en moyenne, le plus de blessés et de morts est la tuerie de masse. Lorsque le suspect connaît les victimes, celles-ci ont plus de chances de mourir que d'être blessées (en moyenne presque 3 morts mais 1 blessé) alors que quand les victimes sont inconnues, celles-ci ont plus de chances d'être blessées que tuées (en moyenne 4 blessés mais 1.4 morts). \nLe deuxième type d'incident par arme à feu provoquant le plus de morts sont les incidents dans le cadre familial. Lorsque la victime et un(e) (ex-)partnaire du suspect, il y a en moyenne 0.7 morts (et 0.3 blessés) et lorsqu'elle fait partie de la famille du suspect, il y a en moyenne 0.8 morts (et 0.4 blessés). \nAinsi, les violences conjugales et intrafamiliales font partie des violences par arme à feu les plus meurtrières, avant les violences entre gangs, par exemple. \nAprès les tueries de masse, les incidents par arme à feu provoquant le plus de blessés sont les violences de gangs (0.8 blessés et 0.47 morts en moyenne) puis les vols à main armée (0.7 blessés et 0.2 morts en moyenne).")

Ce graphique mets en évidence de fortes disparités de conséquences selon les différents types d'incidents par arme à feu, ici caractérisés par les différentes relation du suspect aux victimes. 
En effet, on remarque que le type d'incident provoquant, en moyenne, le plus de blessés et de morts est la tuerie de masse. Lorsque le suspect connaît les victimes, celles-ci ont plus de chances de mourir que d'être blessées (en moyenne presque 3 morts mais 1 blessé) alors que quand les victimes sont inconnues, celles-ci ont plus de chances d'être blessées que tuées (en moyenne 4 blessés mais 1.4 morts). 
Le deuxième type d'incident par arme à feu provoquant le plus de morts sont les incidents dans le cadre familial. Lorsque la victime et un(e) (ex-)partnaire du suspect, il y a en moyenne 0.7 morts (et 0.3 blessés) et lorsqu'elle fait partie de la famille du suspect, il y a en moyenne 0.8 morts (et 0.4 blessés). 
Ainsi, les violences conjugales et intrafamiliales font partie des violences par ar

In [48]:
grouped_data9 = base_totale.groupby('relation').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig9 = px.bar(
    grouped_data9,
    x=grouped_data9['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig9.update_layout(barmode='group')  # Display bars side by side

fig9.show()

In [96]:
print(f"Ce graphique reprenant la même classification des types d'incidents par arme à feu que le précédent permet de se rendre compte des conséquences de ces incidents à l'échelle agrégée plutôt qu'en moyenne. \nLe type d'incidents provoquant le plus de blessés est alors le vol à main armé ({3535/grouped_data12['n_injured'].sum()*100}% des blessés dont la relation avec le suspect est précisée), notamment en raison de leur fréquence très élevée. \nLes violences conjugales et intrafamiliales sont le type d'incident par arme à feu le plus meurtrier ({(3009+2556)/grouped_data12['n_killed'].sum()*100}% des morts dont on connaît la relation au suspect).")

Ce graphique reprenant la même classification des types d'incidents par arme à feu que le précédent permet de se rendre compte des conséquences de ces incidents à l'échelle agrégée plutôt qu'en moyenne. 
Le type d'incidents provoquant le plus de blessés est alors le vol à main armé (40.56690383291255% des blessés dont la relation avec le suspect est précisée), notamment en raison de leur fréquence très élevée. 
Les violences conjugales et intrafamiliales sont le type d'incident par arme à feu le plus meurtrier (65.76459465847317% des morts dont on connaît la relation au suspect).


In [52]:
print("\nV. Les conséquences d'un incident dépendent-elles de l'Etat dans lequel celui-ci s'est déroulé ?\n \nCette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans la partie suivante. \nIntéressons nous d'abord aux nombre de victimes des incidents par arme à feu dans chaque Etat.")


V. Les conséquences d'un incident dépendent-elles de l'Etat dans lequel celui-ci s'est déroulé ?
 
Cette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans la partie suivante. 
Intéressons nous d'abord aux nombre de victimes des incidents par arme à feu dans chaque Etat.


In [53]:
grouped_data10 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig10 = px.bar(
    grouped_data10,
    x=grouped_data10['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig10.update_layout(barmode='group')  # Display bars side by side

fig10.show()

In [41]:
print("Ce graphique mets en évidence les Etats dans lesquels les victimes sont les plus nombreuses. Les Etats dans lesquels il y a eu le plus de blessés sont l'Illinois, le Maryland, la Virginie et le Missouri alors que ceux dans lesquels il y a eu le plus de morts sont la Californie, le Missouri, le Maryland, la Virginie et le Texas. \nOn voit déjà ici que selon l'endroit où un incident par arme à feu se déroule, il n'a pas les mêmes conséquences. Il y a donc des caractéristiques spécifiques aux Etats qui doivent expliquer ces différences. \nOn peut également remarquer que ces Etats ne sont pas exactement les mêmes que ceux dans lesquels il y a le plus d'incidents par arme à feu, un incident ne semble donc pas avoir la même gravité en termes de conséquences partout.")

Ce graphique mets en évidence les Etats dans lesquels les victimes sont les plus nombreuses. Les Etats dans lesquels il y a eu le plus de blessés sont l'Illinois, le Maryland, la Virginie et le Missouri alors que ceux dans lesquels il y a eu le plus de morts sont la Californie, le Missouri, le Maryland, la Virginie et le Texas. 
On voit déjà ici que selon l'endroit où un incident par arme à feu se déroule, il n'a pas les mêmes conséquences. Il y a donc des caractéristiques spécifiques aux Etats qui doivent expliquer ces différences. 
On peut également remarquer que ces Etats ne sont pas exactement les mêmes que ceux dans lesquels il y a le plus d'incidents par arme à feu, un incident ne semble donc pas avoir la même gravité en termes de conséquences partout.


In [54]:
grouped_data11 = base_totale.groupby('state').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig11 = px.bar(
    grouped_data11,
    x=grouped_data11['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Moyenne du nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig11.update_layout(barmode='group')  # Display bars side by side

fig11.show()

In [55]:
print("Même si les disparités entre Etats se sont atténuées par rapport au graphique précédent, ce graphique nous permet tout de même de mettre en évidence des différences en terme de nombre moyen de victimes par incident. \nAinsi, dans l'Illinois, les incidents font en moyenne plus de blessés que dans les autres Etats (presque 0,8 par incident) alors qu'ils font relativement peu de morts (0,18). \nAu contraire, dans certains Etats tels que l'Arizona, les incidents font un nombre moyen de blessés (0.46) mais la moyenne de morts par incident est très élevée (quasiment 0,47 par incident). \nLes taux de mortalité et de 'blessure' d'un incident varient donc beaucoup d'un Etat à l'autre, et ne sont pas nécessairement corrélés. Dans certains Etats, les incidents font peu de morts et peu de blessés (Maine, New Hampshire), \ndans d'autres ils font beaucoup de blessés et de morts (Arizona), dans d'autres encore ils font beaucoup de blessés et peu de morts (Illinois, Maryland) et finalement ils peuvent même faire plus de morts que de blessés en moyenne (Wyoming, Montana, Idaho).")

Même si les disparités entre Etats se sont atténuées par rapport au graphique précédent, ce graphique nous permet tout de même de mettre en évidence des différences en terme de nombre moyen de victimes par incident. 
Ainsi, dans l'Illinois, les incidents font en moyenne plus de blessés que dans les autres Etats (presque 0,8 par incident) alors qu'ils font relativement peu de morts (0,18). 
Au contraire, dans certains Etats tels que l'Arizona, les incidents font un nombre moyen de blessés (0.46) mais la moyenne de morts par incident est très élevée (quasiment 0,47 par incident). 
Les taux de mortalité et de 'blessure' d'un incident varient donc beaucoup d'un Etat à l'autre, et ne sont pas nécessairement corrélés. Dans certains Etats, les incidents font peu de morts et peu de blessés (Maine, New Hampshire), 
dans d'autres ils font beaucoup de blessés et de morts (Arizona), dans d'autres encore ils font beaucoup de blessés et peu de morts (Illinois, Maryland) et finalement ils peuvent mêm

In [56]:
grouped_data12 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum',
    'pop_state_2015':'first'
}).reset_index()
grouped_data12['injured_per_100K']=grouped_data12["n_injured"]/grouped_data12['pop_state_2015']*100
grouped_data12['killed_per_100K']=grouped_data12["n_killed"]/grouped_data12['pop_state_2015']*100

fig12 = px.bar(
    grouped_data12,
    x=grouped_data12['state'],
    y=['injured_per_100K', 'killed_per_100K'],
    labels={"state": "Etat", "value": "nombre de victimes pour 100 000 habitants", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu pour 100 000 habitants par Etat",
    color_discrete_map={"injured_per_100K": "firebrick", "killed_per_100K": "steelblue"},
)

fig12.update_layout(barmode='group')  # Display bars side by side

fig12.show()

In [52]:
print("Ce graphique mets en évidence les Etats ayant le plus de blessés et de tués lors d'incidents par arme à feu pour 100 000 habitants. \nL'Etat District of Columbia est ici un outlier, alors qu'il ne l'était pas dans les autres mesures des conséquences des incidents, ce qui pourrait indiquer une erreur ou simplement une très forte concentration de victimes en comparaison à la taille de l'Etat. \nOn retrouve les Etats du Maryland, Missouri, Virginie et Illinois comme ceux dans lesquels les incidents par arme à feu font le plus de blessés et de morts. \nEn dehors de ces 11 Etats dans lesquels les incidents par arme à feu ont fait plus de 50 blessés pour 100 000  habitants entre 2013 et 2018, les autres sont tous relativement similaires et les incidents qui s'y déroulent ont provoqué entre 5 et 35 blessés pour 100 000 habitants.")

Ce graphique mets en évidence les Etats ayant le plus de blessés et de tués lors d'incidents par arme à feu pour 100 000 habitants. 
L'Etat District of Columbia est ici un outlier, alors qu'il ne l'était pas dans les autres mesures des conséquences des incidents, ce qui pourrait indiquer une erreur ou simplement une très forte concentration de victimes en comparaison à la taille de l'Etat. 
On retrouve les Etats du Maryland, Missouri, Virginie et Illinois comme ceux dans lesquels les incidents par arme à feu font le plus de blessés et de morts. 
En dehors de ces 11 Etats dans lesquels les incidents par arme à feu ont fait plus de 50 blessés pour 100 000  habitants entre 2013 et 2018, les autres sont tous relativement similaires et les incidents qui s'y déroulent ont provoqué entre 5 et 35 blessés pour 100 000 habitants.


In [57]:
grouped_data13 = base_totale.groupby('state').agg({
    'n_killed': 'mean',
    'indic_suicide':'mean'
}).reset_index()


fig13 = px.bar(
    grouped_data13,
    x=grouped_data13['state'],
    y=['n_killed', 'indic_suicide'],
    labels={"state": "Etat", "value": "Moyenne du nb de morts par incident, taux de suicide", "variable": "Décès"},
    title="Comparaison du nombre moyen de morts par incident et du taux de suicide parmi les incidents, par Etat",
    color_discrete_map={"n_killed": "firebrick", "indic_suicide": "steelblue"},
)

fig13.update_layout(barmode='group')  # Display bars side by side

fig13.show()

In [59]:
print(f"La proportion de suicide parmi les incidents varie largement d'un Etat à l'autre, probablement en raison de caractéristiques socio-démographiques et économiques de ces Etats. On voit que cette proportion est la plus élevée dans l'Arizona (0.28), le Nevada, le Texas, l'Utah et l'Idaho. \nCette proportion est fortement corrélée au nombre de morts ({base_totale['indic_suicide'].corr(base_totale['n_killed'])}), ce qui est logique car la réalisation d'un suicide implique au moins un mort, mais elle est également négativement corrélée au nombre de blessés ({base_totale['indic_suicide'].corr(base_totale['n_injured'])}), \nce qui pourrait expliquer, en partie, pourquoi ces Etats précédemment mentionnés font partie de ceux dans lesquels le nombre de blessés et le nombre de morts par arme à feu sont relativement proches, au contraires d'Etats dans lesquels le nombre de blessés est largement supérieur à celui des morts.")

La proportion de suicide parmi les incidents varie largement d'un Etat à l'autre, probablement en raison de caractéristiques socio-démographiques et économiques de ces Etats. On voit que cette proportion est la plus élevée dans l'Arizona (0.28), le Nevada, le Texas, l'Utah et l'Idaho. 
Cette proportion est fortement corrélée au nombre de morts (0.6276074308002891), ce qui est logique car la réalisation d'un suicide implique au moins un mort, mais elle est également négativement corrélée au nombre de blessés (-0.025296986883618654), 
ce qui pourrait expliquer, en partie, pourquoi ces Etats précédemment mentionnés font partie de ceux dans lesquels le nombre de blessés et le nombre de morts par arme à feu sont relativement proches, au contraires d'Etats dans lesquels le nombre de blessés est largement supérieur à celui des morts.
